# Creation of text analysis model
I have seen that the decision tree is the model that offers me the best performance, especially with TfIdf and nltk/regex tokenization with PorterStemmer() stemming.

The code pieces are the same as the code pieces of "analysis.ipynb", but this is needed for cross validation. 

In [2]:
from my_tokenizer import MyTokenizer
import pandas as pd
import util_strings as utils

In [3]:
mt = MyTokenizer(pd.read_csv(utils.completed_dataset, index_col=0))
mt.feature_normalization()

In [4]:
mt.clean_text()
vectorizer = True 
X_train, X_test, y_train, y_test = mt.set_bigram_and_get_sets(vectorizer) 
path_vec = utils.TfidfVectorizer if vectorizer else utils.CountVectorizer

## Classificazione del modello
Migliori risultati con Decision Tree.
Viene salvato il modello

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as mtr

In [6]:
X1, X2, y1, y2 = mt.set_bigram_and_get_sets2(vectorizer) 

matrix_training = mt.vectorizer.fit_transform(X1) #training set
b = mt.vectorizer.transform(X2) #test set

In [16]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(matrix_training, y1)

y_pred = model.predict(b)
report = mtr.classification_report(y2, y_pred, output_dict=True, zero_division=0)
report['accuracy']

0.8098159509202454

Salvataggio del modello

In [ ]:
import pickle
file_name = utils.classificatorTfIdf if vectorizer else utils.classificatorCV_LR
with open(file_name, 'wb') as file:
    pickle.dump(model, file)

# Evaluation of the model
A k-fold cross validation is used to validate the built model
https://scikit-learn.org/stable/modules/cross_validation.html

In [17]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import cross_validate

k_folds = KFold(n_splits = 10)
scores = cross_val_score(model, matrix_training, y1, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.61832061 0.66153846 0.76153846 0.83076923 0.84615385 0.79230769
 0.76153846 0.76923077 0.78461538 0.76923077]
Average CV Score:  0.7595243687610099
Number of CV Scores used in Average:  10


In [75]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth':  [2, 4, 6, 8, 10, 12],
    'min_samples_split': [2,3,4],
    'min_samples_leaf': [1,2]
}

clf = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=params,
    cv=5,
    n_jobs=5,
    verbose=1,
)

clf.fit(X_train, y_train)
model = clf.best_params_

model.f

Fitting 5 folds for each of 36 candidates, totalling 180 fits


dict